# Bank Marketing

## Load dataset

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [7]:
df = pd.read_csv('https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv')

In [8]:
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [9]:
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [10]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [11]:
df.nunique()

lead_source                    5
industry                       7
number_of_courses_viewed      10
annual_income               1267
employment_status              4
location                       7
interaction_count             12
lead_score                   101
converted                      2
dtype: int64

In [12]:
categorical = ['lead_source','industry', 'employment_status','location']

In [13]:
numerical   = ['number_of_courses_viewed','annual_income','interaction_count', 'lead_score']

In [14]:
df[categorical] = df[categorical].fillna('NA')

In [15]:
df[numerical] = df[numerical].fillna(0)

In [16]:
{col:df[col].unique() for col in categorical}

{'lead_source': array(['paid_ads', 'social_media', 'events', 'referral', 'organic_search',
        'NA'], dtype=object),
 'industry': array(['NA', 'retail', 'healthcare', 'education', 'manufacturing',
        'technology', 'other', 'finance'], dtype=object),
 'employment_status': array(['unemployed', 'employed', 'NA', 'self_employed', 'student'],
       dtype=object),
 'location': array(['south_america', 'australia', 'europe', 'africa', 'middle_east',
        'NA', 'north_america', 'asia'], dtype=object)}

In [17]:
df['industry'].value_counts()

industry
retail           203
finance          200
other            198
healthcare       187
education        187
technology       179
manufacturing    174
NA               134
Name: count, dtype: int64

## Correlation matrix

In [18]:
 corr_matrix = df[numerical].corr()

In [19]:
corr_matrix

,number_of_courses_viewed,annual_income,interaction_count,lead_score
number_of_courses_viewed,1.000000,0.009770,-0.023565,-0.004879
annual_income,0.009770,1.000000,0.027036,0.015610
interaction_count,-0.023565,0.027036,1.000000,0.009888
lead_score,-0.004879,0.015610,0.009888,1.000000


## Prepare the training set 

In [20]:
from sklearn.model_selection import train_test_split

In [21]:
full_train, test_set = train_test_split(df , test_size=0.2, random_state= 42)
train_set, val_set   = train_test_split(full_train, test_size=0.25, random_state= 42)

In [22]:
train_set = train_set.reset_index(drop=True)
val_set   = val_set.reset_index(drop=True)
test_set  = test_set.reset_index(drop=True)

In [23]:
y_train = train_set['converted']
y_val = val_set['converted']
y_test = test_set['converted']

In [24]:
del train_set['converted']
del val_set['converted']
del test_set['converted']

## Mutual Information

In [25]:
from sklearn.metrics import mutual_info_score

In [26]:
def mutual_info_converted(series):
    return mutual_info_score(series, y_train)

In [27]:
train_set[categorical].apply(mutual_info_converted).sort_values(ascending=False)

lead_source          0.035396
employment_status    0.012938
industry             0.011575
location             0.004464
dtype: float64

## One hot encoder

In [28]:
from sklearn.feature_extraction import DictVectorizer

In [29]:
dv = DictVectorizer()

In [30]:
train_set

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score
0,paid_ads,retail,0,58472.0,student,middle_east,5,0.03
1,organic_search,manufacturing,3,71738.0,student,middle_east,6,0.77
2,paid_ads,technology,3,81973.0,employed,north_america,2,0.59
3,NA,technology,1,74956.0,employed,europe,3,0.34
4,organic_search,retail,3,59335.0,student,australia,1,0.98
...,...,...,...,...,...,...,...,...
871,organic_search,other,1,43907.0,employed,australia,4,0.33
872,social_media,retail,3,64969.0,employed,north_america,1,0.18
873,NA,education,3,89042.0,employed,asia,4,0.75
874,social_media,manufacturing,1,0.0,self_employed,europe,1,0.65


In [31]:
train_dicts = train_set[list[categorical , numerical]].to_dict(orient='records')

In [32]:
dv = DictVectorizer(sparse=False)

In [33]:
dv.fit(train_dicts)

,dtype,<class 'numpy.float64'>
,separator,'='
,sparse,False
,sort,True


In [34]:
dv.get_feature_names_out()

array(['annual_income', 'employment_status=NA',
       'employment_status=employed', 'employment_status=self_employed',
       'employment_status=student', 'employment_status=unemployed',
       'industry=NA', 'industry=education', 'industry=finance',
       'industry=healthcare', 'industry=manufacturing', 'industry=other',
       'industry=retail', 'industry=technology', 'interaction_count',
       'lead_score', 'lead_source=NA', 'lead_source=events',
       'lead_source=organic_search', 'lead_source=paid_ads',
       'lead_source=referral', 'lead_source=social_media', 'location=NA',
       'location=africa', 'location=asia', 'location=australia',
       'location=europe', 'location=middle_east',
       'location=north_america', 'location=south_america',
       'number_of_courses_viewed'], dtype=object)

In [35]:
X_train   = dv.transform(train_dicts)

In [36]:
val_dicts = val_set[list[categorical , numerical]].to_dict(orient='records')

In [37]:
X_val     = dv.transform(val_dicts)

## Train The Model

In [38]:
from sklearn.linear_model import LogisticRegression

In [39]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)

In [40]:
model.fit(X_train,y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'liblinear'
,max_iter,1000
,multi_class,'deprecated'


In [41]:
y_pred = model.predict(X_val)

In [53]:
accuracy =(y_val == y_pred).mean()

## Feature Elimination

In [48]:
all_features= categorical + numerical

In [56]:
for col in all_features:
    columns = all_features.copy()
    columns.remove(col)
    train_dict = train_set[columns].to_dict(orient= 'records')
    X_train_features = dv.fit_transform(train_dict)
    val_dict = val_set[columns].to_dict(orient= 'records')
    X_val_features = dv.fit_transform(val_dict)
    model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    model.fit(X_train_features,y_train)
    y_pred_feature = model.predict(X_val_features)
    print(col)
    print((y_val == y_pred_feature).mean()- accuracy )
    

lead_source
0.0030716723549488734
industry
-0.0003412969283276279
employment_status
-0.003754266211604018
location
0.009897610921501765
number_of_courses_viewed
-0.14368600682593857
annual_income
0.1532423208191127
interaction_count
-0.14368600682593857
lead_score
0.006484641638225264


## With regularization

In [57]:
C =[0.01, 0.1, 1, 10, 100]

In [58]:
for c in C :
    model = LogisticRegression(solver='liblinear', C=c, max_iter=1000, random_state=42)
    model.fit(X_train,y_train)
    y_pred = model.predict(X_val)
    print(c)
    print((y_pred == y_val).mean())

0.01
0.6996587030716723
0.1
0.6996587030716723
1
0.6996587030716723
10
0.6996587030716723
100
0.6996587030716723
